In [1]:
from backtesting import set_bokeh_output
set_bokeh_output(notebook=False)

# !python3 -m pip install bokeh==2.4.3

In [2]:
import sys
from datetime import timedelta

In [3]:
# sys.path.insert(1, '/home/kenny/algotrading/model_training')
sys.path.insert(1, "/Users/arsenchik/Desktop/dipploma/machine_learning_in_hft/algotrading/src/backtesting")
sys.path.insert(1, '/Users/arsenchik/Desktop/dipploma/machine_learning_in_hft/algotrading/src/models')
sys.path.insert(1, '/Users/arsenchik/Desktop/dipploma/machine_learning_in_hft/algotrading/src/configs')
sys.path.insert(1, "/Users/arsenchik/Desktop/dipploma/machine_learning_in_hft/algotrading/src/data")

In [4]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from tqdm import tqdm
from backtesting import Backtest, Strategy
from sqlalchemy import create_engine, text

from strategies import MultipleBasic, SmaCross, BinaryBasic, Multiple, Binary, Random
from config import *
from dataset import Data
from experiments import EXPERIMENT_ID_TO_FEATURES

In [5]:
url = DATABASE_URL_1MINUTE
td_days = 0

In [6]:
engine = create_engine(url=url)

with engine.connect() as connection:
        # query = "SELECT name FROM sqlite_schema WHERE type='table'"
        # symbols = [item[0] for item in connection.execute(text(query)).fetchall()]
        # for symbol in symbols:
        symbol = 'BTCUSDT'
        qry = f"SELECT * FROM '{symbol}' WHERE Open_Time < '{pd.to_datetime('today') - timedelta(days = td_days)}'"
        data = pd.DataFrame(connection.execute(text(qry)))
        data['Open_Time'] = pd.to_datetime(data['Open_Time'])
        data.set_index("Open_Time", inplace=True)
        # data.index = pd.to_datetime(data.index)
        # data.drop(['Close_Time'], axis=1, inplace=True)
connection.close()

## Random classifier

In [7]:
cash = 100000
commission = .0001
bt_random = Backtest(data, Random, cash=cash, commission=commission)
output = bt_random.run()

In [8]:
output

Start                     2023-02-22 06:08:00
End                       2023-02-27 06:07:00
Duration                      4 days 23:59:00
Exposure Time [%]                   77.791667
Equity Final [$]                 92546.902942
Equity Peak [$]                      100000.0
Return [%]                          -7.453097
Buy & Hold Return [%]               -2.963834
Return (Ann.) [%]                  -98.126239
Volatility (Ann.) [%]                0.408417
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -7.490269
Avg. Drawdown [%]                   -7.490269
Max. Drawdown Duration        4 days 23:58:00
Avg. Drawdown Duration        4 days 23:58:00
# Trades                                 2759
Win Rate [%]                        41.536789
Best Trade [%]                       0.543709
Worst Trade [%]                     -1.089354
Avg. Trade [%]                    

In [9]:
bt_random.plot()

Row(id='1475', ...)

## Binary LightCoin strategy

In [5]:
binary_model_path_txt = '/Users/arsenchik/Desktop/dipploma/machine_learning_in_hft/algotrading/model_training/models/binary_model_1_min.txt'

binary_model = lgb.Booster(model_file=binary_model_path_txt)

features = EXPERIMENT_ID_TO_FEATURES[24]
test_data = Data(data)
test_data.create_features()
test_data.df['predicted_proba'] = binary_model.predict(test_data.df[features])

In [6]:
cash = 100000
commission = .0001

bt = Backtest(test_data.df, Binary, cash=cash, commission=commission)
output = bt.run()
# stats = bt.optimize(price_delta=[0.002, 0.004, 0.001],
#                   maximize='Sharpe Ratio')

In [7]:
output

Start                     2023-02-22 06:08:00
End                       2023-02-27 06:07:00
Duration                      4 days 23:59:00
Exposure Time [%]                   83.861111
Equity Final [$]                100636.898432
Equity Peak [$]                 101267.404378
Return [%]                           0.636898
Buy & Hold Return [%]               -2.963834
Return (Ann.) [%]                   54.337715
Volatility (Ann.) [%]               12.011848
Sharpe Ratio                         4.523676
Sortino Ratio                       18.265706
Calmar Ratio                        54.923605
Max. Drawdown [%]                   -0.989333
Avg. Drawdown [%]                   -0.130465
Max. Drawdown Duration        1 days 22:31:00
Avg. Drawdown Duration        0 days 04:17:00
# Trades                                   27
Win Rate [%]                        55.555556
Best Trade [%]                       3.546073
Worst Trade [%]                     -1.336301
Avg. Trade [%]                    

In [8]:
bt.plot()

Row(id='1474', ...)

## Multiple LigthCoin strategy

In [5]:
cash = 100000
commission = .0001

bt = Backtest(data, Multiple, cash=cash, commission=commission)
output = bt.run()
# stats = bt.optimize(price_delta=[0.002, 0.004, 0.001],
#                   maximize='Sharpe Ratio')

In [6]:
output

Start                     2023-02-22 06:08:00
End                       2023-02-27 06:07:00
Duration                      4 days 23:59:00
Exposure Time [%]                   32.083333
Equity Final [$]                100519.904699
Equity Peak [$]                 100701.760357
Return [%]                           0.519905
Buy & Hold Return [%]               -2.963834
Return (Ann.) [%]                   30.263239
Volatility (Ann.) [%]               10.201341
Sharpe Ratio                         2.966594
Sortino Ratio                        11.09004
Calmar Ratio                        37.399474
Max. Drawdown [%]                   -0.809189
Avg. Drawdown [%]                   -0.156033
Max. Drawdown Duration        2 days 14:03:00
Avg. Drawdown Duration        0 days 07:58:00
# Trades                                   28
Win Rate [%]                        35.714286
Best Trade [%]                       2.769027
Worst Trade [%]                     -0.304868
Avg. Trade [%]                    

In [7]:
bt.plot()

Row(id='1475', ...)

In [5]:
cash = 100000
commission = .0001
bt = Backtest(data, Binary, cash=cash, commission=commission)
output = bt.run()

In [6]:
output

Start                     2023-02-22 06:08:00
End                       2023-02-27 06:07:00
Duration                      4 days 23:59:00
Exposure Time [%]                    0.097222
Equity Final [$]                100007.827219
Equity Peak [$]                 100007.827219
Return [%]                           0.007827
Buy & Hold Return [%]               -2.963834
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.018607
Avg. Drawdown [%]                   -0.018607
Max. Drawdown Duration        0 days 00:08:00
Avg. Drawdown Duration        0 days 00:08:00
# Trades                                    7
Win Rate [%]                        71.428571
Best Trade [%]                       0.025829
Worst Trade [%]                     -0.040295
Avg. Trade [%]                    

In [7]:
bt.plot()

Row(id='1474', ...)

In [8]:
output

Start                     2023-02-22 06:08:00
End                       2023-02-27 06:07:00
Duration                      4 days 23:59:00
Exposure Time [%]                    9.194444
Equity Final [$]                 98472.648418
Equity Peak [$]                      100000.0
Return [%]                          -1.527352
Buy & Hold Return [%]               -2.963834
Return (Ann.) [%]                  -50.102855
Volatility (Ann.) [%]                  0.9343
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -1.527352
Avg. Drawdown [%]                   -1.527352
Max. Drawdown Duration        4 days 23:46:00
Avg. Drawdown Duration        4 days 23:46:00
# Trades                                  432
Win Rate [%]                        44.907407
Best Trade [%]                       0.049496
Worst Trade [%]                     -0.067931
Avg. Trade [%]                    

In [8]:
output

Start                     2023-02-22 06:08:00
End                       2023-02-27 06:07:00
Duration                      4 days 23:59:00
Exposure Time [%]                   62.291667
Equity Final [$]                 92527.550636
Equity Peak [$]                 100102.361703
Return [%]                          -7.472449
Buy & Hold Return [%]               -2.963834
Return (Ann.) [%]                  -94.390044
Volatility (Ann.) [%]                0.820879
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -7.567065
Avg. Drawdown [%]                   -1.526705
Max. Drawdown Duration        4 days 23:32:00
Avg. Drawdown Duration        0 days 23:58:00
# Trades                                 2247
Win Rate [%]                        46.283934
Best Trade [%]                        0.07142
Worst Trade [%]                     -0.096647
Avg. Trade [%]                    

In [9]:
bt.plot()

Row(id='1475', ...)